In [7]:
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

filepath = 'data/ss2.png'
image = Image.open(filepath).convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decod

In [32]:
generated_ids = model.generate(pixel_values, max_new_tokens=1)

tensor([[    2, 34603,   195,    12,   698, 10437, 28975,    36,   683,    38,
           120,  4839,     2]])


In [ ]:
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

In [2]:
!pip install quanto

  Obtaining dependency information for quanto from https://files.pythonhosted.org/packages/06/22/89e363878024a21b0093f69c7040c33715ff52cd0cbfb1ac91a0977f511f/quanto-0.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig

quantization_config = QuantoConfig(load_in_8bit=True)



# Load the LLaMA model and tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
llama_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config=quantization_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
llama_model.device

device(type='mps', index=0)

In [22]:
# Feed the current token to LLaMA
prompt = f"Generate the next token after: {generated_text}"
input_ids = llama_tokenizer(prompt, return_tensors="pt").input_ids.to(llama_model.device)

# Generate the next token
llama_output = llama_model.generate(input_ids, max_new_tokens=1)
predicted_token = llama_tokenizer.decode(llama_output[0][-2:], skip_special_tokens=True)
print("Predicted token from LLaMA:", predicted_token)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted token from LLaMA:  Test the


In [23]:
# Add the token predicted by LLaMA to TrOCR's decoder input
decoder_input_ids = processor.tokenizer(predicted_token, return_tensors="pt").input_ids
generated_ids = model.generate(
    pixel_values,
    decoder_input_ids=decoder_input_ids,
    max_new_tokens=1
)
next_token = processor.batch_decode(generated_ids, skip_special_tokens=True)[-1]
print("Next token predicted by TrOCR:", next_token)

Next token predicted by TrOCR:  Test the valuation


In [33]:
import numpy as np

vocab_size = 50265

all_ids = np.arange(vocab_size).reshape(1, -1)

# Get the corresponding text for each token ID
text = []
for id in all_ids[0]:
    text.append(processor.tokenizer.decode(id))

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 '.',
 ' the',
 ',',
 ' to',
 ' and',
 ' of',
 ' a',
 ' in',
 '-',
 ' for',
 ' that',
 ' on',
 ' is',
 '�',
 "'s",
 ' with',
 ' The',
 ' was',
 ' "',
 ' at',
 ' it',
 ' as',
 ' said',
 '�',
 ' be',
 's',
 ' by',
 ' from',
 ' are',
 ' have',
 ' has',
 ':',
 ' (',
 ' he',
 ' I',
 ' his',
 ' will',
 ' an',
 ' this',
 ')',
 ' �',
 ' not',
 '�',
 ' you',
 '�',
 ' their',
 ' or',
 ' they',
 ' we',
 ' but',
 ' who',
 ' more',
 ' had',
 ' been',
 ' were',
 ' about',
 ',"',
 ' which',
 ' up',
 ' its',
 ' can',
 ' one',
 ' out',
 ' also',
 ' $',
 ' her',
 ' all',
 ' after',
 '."',
 '/',
 ' would',
 "'t",
 ' year',
 ' when',
 ' first',
 ' she',
 ' two',
 ' over',
 ' people',
 ' A',
 ' our',
 ' It',
 ' time',
 ' than',
 ' into',
 ' there',
 't',
 ' He',
 ' new',
 ' —',
 ' last',
 ' just',
 ' In',
 ' other',
 ' so',
 ' what',
 'I',
 ' like',
 'a',
 ' some',
 'S',
 'ë',
 ' them',
 ' years',
 "'",
 ' do',
 ' your',
 ' -',
 ' 1',
 '"',
 ' if',
 ' could',
 '?',
 ' n

In [37]:
# Get text for all llama tokens

llama_vocab_size = llama_tokenizer.vocab_size

llama_all_ids = np.arange(llama_vocab_size).reshape(1, -1)

llama_text = []
for id in llama_all_ids[0]:
    llama_text.append(llama_tokenizer.decode(id))

llama_text[300:340]

['as',
 'el',
 'ct',
 'nd',
 ' in',
 ' h',
 'ent',
 'id',
 ' n',
 'am',
 '           ',
 ' to',
 ' re',
 '--',
 ' {',
 ' of',
 'om',
 ');\n',
 'im',
 '\r\n',
 ' (',
 'il',
 '//',
 ' and',
 'ur',
 'se',
 ' l',
 'ex',
 ' S',
 'ad',
 ' "',
 'ch',
 'ut',
 'if',
 '**',
 ' }',
 'em',
 'ol',
 '                ',
 'th']